# Robot localization and Particle Filters

Import libraries and load map

In [1]:
import numpy as np
import cv2

map = cv2.imread('map.jpeg', 0)

print(map)

# inisilize robot postion
HEIGHT, WIDTH = map.shape

rx, ry, rtheta = (WIDTH / 4, HEIGHT / 4, 0)

[[ 92  87  97 ...  86 123 104]
 [ 93  87  97 ... 120 116  99]
 [ 94  88  97 ... 113 101 106]
 ...
 [109 107 112 ...  91  79  73]
 [104 107 112 ...  95  90 104]
 [103 111 115 ...  54  63 105]]


# Read keyboard input 

In [3]:
STEP = 5
TURN = np.radians(25)

def get_input():

    fwd = 0
    turn = 0
    halt = False

    k = cv2.waitKey(0)
    if k == 82: # up 
        fwd = STEP 
    elif k == 83: # right
        turn = TURN
    elif k == 81: # left 
        turn = -TURN
    else:
        halt = True

    return fwd, turn, halt

In [4]:
SIGMA_STEP = 5
SIGMA_TURN = np.radians(5)

def move_robot(rx, ry, rtheta, fwd, turn):

    fwd_noisy = np.random.normal(fwd, SIGMA_STEP, 1)
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)

    print('fwd noisy = ', fwd_noisy)
    turn_noisy = np.random.normal(turn, SIGMA_TURN, 1)

    rtheta += turn_noisy
    print('turn noisy = ', np.degrees(turn_noisy))

    return rx, ry, rtheta

In [5]:
NUM_PARTICLES = 3000 # initialize number of particles

def init():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles *= np.array((WIDTH, HEIGHT, np.radians(360)))
    return particles

In [6]:
def move_particles(particles, fwd, turn):

    particles[:, 0] += fwd * np.cos(particles[:, 2])
    particles[:, 1] += fwd * np.sin(particles[:, 2])
    particles[:, 2] += turn

    particles[:, 0] = np.clip(particles[:, 0], 0.0, WIDTH - 1)
    particles[:, 1] = np.clip(particles[:, 1], 0.0, HEIGHT - 1)
    return particles

In [7]:
SIGMA_SENSOR = 2 # two pixel

def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)

    if noisy:
        return np.random.normal(map[y,x], SIGMA_SENSOR, 1)
    return map[y, x]

In [8]:
def compute_weights(particles, robot_sensor):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):

        elevation = sense(particles[i, 0], particles[i, 1], noisy=False)
        errors[i] = abs(robot_sensor - elevation)
        
    weights = np.max(errors) - errors

    weights[
        (particles[:, 0] == 0) |
        (particles[:, 0] == WIDTH - 1) | 
        (particles[:, 1] == 0) |
        (particles[:, 1] == HEIGHT - 1)
    ] = 0.0
    weights = weights ** 3

    return weights

In [9]:
def resample(particles, weights):
    probabiliies = weights / np.sum(weights)
    new_index = np.random.choice(
      NUM_PARTICLES, 
      size=NUM_PARTICLES,
      p=probabiliies
    )
    particles = particles[new_index, :]
    return particles

In [10]:
SIGMA_POS = 2 
SIGMA_TURN = np.radians(10)

def add_noise(particles):
    noise = np.concatenate((
      np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
      np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
      np.random.normal(0, SIGMA_TURN, (NUM_PARTICLES, 1))
    ),
    axis=1
    )

    particles += noise
    return particles

In [11]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)

    # display particles 

    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, (int(particles[i, 0]), int(particles[i, 1])), 1, (255, 0, 0), 1)

    # display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0, 255, 0), 10)

    # display best guess

    if len(particles) > 0:
        px = np.mean(particles[:, 0])
        py = np.mean(particles[:, 1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0, 0, 255), 5)

    cv2.imshow('map', lmap)


In [12]:
particles = init()

while True:

    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)

    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)


cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x2106430) is not the object's thread (0x1e63560).
Cannot move to target thread (0x2106430)

QObject::moveToThread: Current thread (0x2106430) is not the object's thread (0x1e63560).
Cannot move to target thread (0x2106430)

QObject::moveToThread: Current thread (0x2106430) is not the object's thread (0x1e63560).
Cannot move to target thread (0x2106430)

QObject::moveToThread: Current thread (0x2106430) is not the object's thread (0x1e63560).
Cannot move to target thread (0x2106430)

QObject::moveToThread: Current thread (0x2106430) is not the object's thread (0x1e63560).
Cannot move to target thread (0x2106430)

QObject::moveToThread: Current thread (0x2106430) is not the object's thread (0x1e63560).
Cannot move to target thread (0x2106430)

QObject::moveToThread: Current thread (0x2106430) is not the object's thread (0x1e63560).
Cannot move to target thread (0x2106430)

QObject::moveToThread: Current thread (0x2106430) is not the object's thread

fwd noisy =  [0.3748704]
turn noisy =  [-7.43204413]
fwd noisy =  [-2.37167264]
turn noisy =  [-8.85263698]
fwd noisy =  [-1.21494805]
turn noisy =  [5.5096805]
fwd noisy =  [2.61989506]
turn noisy =  [29.38456951]
fwd noisy =  [-0.13592622]
turn noisy =  [2.01394874]
